In [1]:
# only adjust settings in this cell
state = 'KS' 
year = 2020
enumeration_limit = 10

In [2]:
import sys, os
src_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(src_path)

In [3]:
filepath = '../../dat/' + str(year) + '/'
filename = state + '_county.json'
filename2 = state + '_county.shp'

In [4]:
from read import read_graph_from_json
import networkx as nx

G = read_graph_from_json(state, filepath + filename, year=year)
print(f"The state of {state} has {G._k} districts.")
ideal_population = sum(G.nodes[i]['TOTPOP'] for i in G.nodes) / G._k

The state of KS has 4 districts.


In [ ]:
from optimization import iterative_refinement
from math import ceil, floor
from metrics import *
import time

deviation = 1/2
max_deviation = 0.01 * ideal_population
plans = list()
first_feasible_dev = None

while True:

    print("*****************************************")
    print(f"Trying deviation = {deviation}.")
    print("*****************************************")
    
    L = ceil( ideal_population - deviation )
    U = floor( ideal_population + deviation )

    start_time = time.perf_counter()
    new_plans = iterative_refinement(G, L, U, G._k, state, enumeration_limit=enumeration_limit, verbose=False)
    print("Total time =",round(time.perf_counter() - start_time,2))
    
    if new_plans:
        plans += new_plans
        if first_feasible_dev is None:
            min_observed_dev = min(observed_deviation_persons(G, plan, ideal_population) for plan in new_plans)
            first_feasible_dev = min_observed_dev
            

    if deviation == max_deviation:
        break
    
    deviation *= 2
    deviation = min( deviation, max_deviation )
       

*****************************************
Trying deviation = 0.5.
*****************************************
Set parameter Username
Set parameter LicenseID to value 2608266
Academic license - for non-commercial use only - expires 2026-01-09


In [ ]:
infeasible_region = [0, first_feasible_dev] if first_feasible_dev > 0.5 else None
print(f"Infeasible region is: {infeasible_region}")

In [ ]:
from pareto import ParetoFrontier

# set dummy values... (because compute_obj checks plan feasibility...)
G._L = 0 
G._U = G._k * ideal_population
pareto = dict()

obj_types = ['cut_edges', 'perimeter', 'inverse_Polsby_Popper', 'average_Polsby_Popper', 'bottleneck_Polsby_Popper']

for obj_type in obj_types:

    print("***************************************")
    print("obj_type =", obj_type)
    print("***************************************")
    
    senses = ['min', 'max' if obj_type in ['average_Polsby_Popper', 'bottleneck_Polsby_Popper'] else 'min']
    obj_names = ['deviation_persons', obj_type]
    pareto[obj_type] = ParetoFrontier(senses, obj_names, state=state, level='county')
    
    for plan in plans:  
        dev = observed_deviation_persons(G, plan, ideal_population)
        obj = compute_obj(G, plan, obj_type)
        objs_val = [dev, obj]
        pareto[obj_type].add_plan(plan, upper_bound=objs_val)

    print("Pareto front objective values:", pareto[obj_type].upper_bounds)
    if infeasible_region[1] > 0.5:
        upper_bounds = pareto[obj_type].upper_bounds
        max_dev = max(upper_bound[0] for upper_bound in upper_bounds)
        max_obj = max(upper_bound[1] for upper_bound in upper_bounds)
        min_obj = min(upper_bound[1] for upper_bound in upper_bounds)
        o1lim=[-1, max_dev+100]
        o2lim=[min_obj*0.9, max_obj*(1.1)]
    else:
        o1lim = None
        o2lim = None
    
    pareto[obj_type].tighten_lower_bounds()       
    pareto[obj_type].plot_with_custom_x_ranges(method ='heuristic', splits=None,
                                              o1lim=o1lim, o2lim=o2lim, infeasible_region = infeasible_region)    
    pareto[obj_type].draw_plans(G, filepath, filename2)

In [ ]:
len(plans)

In [ ]:
for obj_type in obj_types:

    print("***************************************")
    print("obj_type =", obj_type)
    print("***************************************")
    print("Pareto front plans:", pareto[obj_type].plans)